In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pompy.models
import pompy.processors

import matplotlib.animation as animation
Writer = animation.FFMpegWriter(fps=30, codec='libx264') # Or 
Writer = animation.FFMpegWriter(fps=20, metadata=dict(artist='Me'), bitrate=1800)
%matplotlib inline
from matplotlib.animation import FuncAnimation
plt.rcParams['animation.html'] = 'html5'

In [2]:

def set_up_figure(fig_size=(10, 5)):
    """Set up Matplotlib figure with simulation time title text.

    Parameters
    ----------
    title_text : string
        Text to set figure title to.
    fig_size : tuple
        Figure dimensions in inches in order `(width, height)`.
    """
    fig, ax = plt.subplots(1, 1, figsize=fig_size)
    title = ax.set_title('Simulation time = ---- seconds')
    ax.set_xlim([-0.5,0.5])
    ax.set_ylim([-0.2,0.2])
    return fig, ax, title

In [3]:
def update_decorator(dt, title, steps_per_frame, models):
    """Decorator for animation update methods."""
    def inner_decorator(update_function):
        def wrapped_update(i):
            for j in range(steps_per_frame):
                for model in models:
                    model.update(dt)
            t = i * steps_per_frame * dt
            title.set_text('Simulation time = {0:.3f} seconds'.format(t))
            return [title] + update_function(i)
        return wrapped_update
    return inner_decorator

In [24]:
def plume_model_demo(dt=0.005, t_max=5, steps_per_frame=1,
                     seed=1):
    """Set up plume model and animate puffs overlayed over velocity field.

    Puff positions displayed using Matplotlib `scatter` plot function and
    velocity field displayed using `quiver` plot function.
    plot and quiver functions.

    Parameters
    ----------
    dt : float
        Simulation timestep.
    t_max : float
        End time to simulate to.
    steps_per_frame: integer
        Number of simulation time steps to perform between animation frames.
    seed : integer
        Seed for random number generator.

    Returns
    -------
    fig : Figure
        Matplotlib figure object.
    ax : AxesSubplot
        Matplotlib axis object.
    anim : FuncAnimation
        Matplotlib animation object.
    """
    
    # define simulation region
    wind_region = pompy.models.Rectangle(x_min=-1., x_max=10, y_min=-2., y_max=2.)
    sim_region = pompy.models.Rectangle(x_min=-1, x_max=1, y_min=-1, y_max=1)
    # set up wind model
    wind_vel_x_av =2
    wind_vel_y_av =0.
    wind_model = pompy.models.WindModel(wind_region, 21, 11, wind_vel_x_av,wind_vel_y_av)

    centre_rel_diff_scale = 2
    puff_release_rate=10
    puff_init_rad=0.001
    
    plume_model = pompy.models.PlumeModel(
        sim_region, (-0.5, 0., 0.), wind_model
    , centre_rel_diff_scale ,
    puff_release_rate,
    puff_init_rad,)
    # set up figure window
    fig, ax, title = set_up_figure()
    # create quiver plot of initial velocity field
    # quiver expects first array dimension (rows) to correspond to y-axis
    # therefore need to transpose
    vf_plot = plt.quiver(
        wind_model.x_points, wind_model.y_points,
        wind_model.velocity_field.T[0], wind_model.velocity_field.T[1],
        width=0.003)
    
    # expand axis limits to make vectors at boundary of field visible
    ax.axis(ax.axis() + np.array([-0.25, 0.25, -0.25, 0.25]))
    
    

    
    # draw initial puff positions with scatter plot
    radius_mult = 2000
    pp_plot = plt.scatter(
        plume_model.puff_array[:, 0], plume_model.puff_array[:, 1],
        radius_mult * plume_model.puff_array[:, 3]**0.5, c='g',
        edgecolors='none')
    
    pp_plot2 = plt.scatter(
        -0.5, 0,
        200, c='r',
        edgecolors='k')
    
    
    ax.set_xlabel('x-coordinate / m')
    ax.set_ylabel('y-coordinate / m')
    ax.set_aspect(1)
    fig.tight_layout()

    # define update function
    @update_decorator(dt, title, steps_per_frame, [wind_model, plume_model])
    def update(i):
        # update velocity field quiver plot data
        vf_plot.set_UVC(wind_model.velocity_field[:, :, 0].T,
                        wind_model.velocity_field[:, :, 1].T)
        # update puff position scatter plot positions and sizes
        pp_plot.set_offsets(plume_model.puff_array[:, :2])
        pp_plot._sizes = radius_mult * plume_model.puff_array[:, 3]**0.5
        return [vf_plot, pp_plot,pp_plot2]

    # create animation object
    n_frame = int(t_max / (dt * steps_per_frame) + 0.5)
    anim = FuncAnimation(fig, update, frames=n_frame, blit=True)
    return fig, ax, anim

In [25]:
fig, ax, anim = plume_model_demo()
plt.close(fig)
anim